In [1]:
#import libraries
import tensorflow as tf
from tensorflow.keras import models,layers
import matplotlib.pyplot as plt

C:\Users\klpna\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [42]:
train_dir = tf.keras.preprocessing.image_dataset_from_directory(
    "../data/train",
    shuffle=True,
    image_size=(256, 256),
    batch_size=32
)


Found 5216 files belonging to 2 classes.


In [43]:
test_dir = tf.keras.preprocessing.image_dataset_from_directory(
    "../data/test",
    shuffle=True,
    image_size=(256, 256),
    batch_size=32
)

Found 624 files belonging to 2 classes.


In [44]:
val_dir = tf.keras.preprocessing.image_dataset_from_directory(
    "../data/val",
    shuffle=True,
    image_size=(256, 256),
    batch_size=32
)

Found 47 files belonging to 2 classes.


In [45]:
class_names = train_dir.class_names
class_names

['NORMAL', 'PNEUMONIA']

In [46]:
len(dataset)

163

In [47]:
163*32

5216

In [49]:
for image_batch,label_batch in train_dir.take(1):
    for i in range(12):
        ax=plt.subplot(3,4,i+1)
        plt.imshow(image_batch[0].numpy().astype("uint8"))
        plt.title(class_names[label_batch[i]])
        plt.axis("off")

NameError: name 'datase' is not defined

In [36]:
len(dataset)

163

In [54]:
train_dir = train_dir.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_dir = val_dir.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_dir = test_dir.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)